In [1]:
'''
Compute prm scores for each question instead of multiple questions
'''

import os, psutil, gc
import time 
import json
import pprint

from collections import defaultdict
import random

In [2]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer
from vllm import LLM, SamplingParams, PoolingParams

from sal.config import Config
from sal.models.reward_models import PRM
from sal.utils.score import score, aggregate_scores

from datasets import Dataset, load_dataset

from core.reward_models import RLHFFlow

from core import best_of_n
from utils.load_data import load_data_prm800k

from utils import load_data

In [3]:
# base_dir
base_dir = '/groups/kjun/tnn/datasets/'

# dataset path
data_dir = base_dir + "/prm800k/math_splits"

# llm and prm path
llm_dir = base_dir + "/Llama-3.2-1B-Instruct-GGUF/Llama-3.2-1B-Instruct.Q4_K_M.gguf"
prm_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data-GGUF/Llama3.1-8B-PRM-Deepseek-Data.Q4_K_M.gguf"

llm_tokenizer_dir = base_dir + "/Llama-3.2-1B-Instruct"
prm_tokenizer_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data"

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
if torch.cuda.is_available():
    GPUs = os.environ.get('CUDA_VISIBLE_DEVICES', "0").split(',')
    print(f"GPUs = {GPUs}")
else:
    print("CUDA is not available.") 

GPUs = ['0', '1', '2', '3']


In [5]:
prm = RLHFFlow(model_path=prm_tokenizer_dir, device_map='cuda:3')

gc.collect();torch.cuda.empty_cache();
print('#--- memory:', torch.cuda.memory_allocated(0)/(1024**3))
print('#--- memory:', torch.cuda.memory_allocated(1)/(1024**3))
# print('#--- memory:', torch.cuda.memory_allocated(2)/(1024**3))
# print('#--- memory:', torch.cuda.memory_allocated(3)/(1024**3))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

#--- memory: 0.0
#--- memory: 0.0


In [6]:
# del(prm)
gc.collect();torch.cuda.empty_cache();
print('#--- memory:', torch.cuda.memory_allocated(0)/(1024**3))
print('#--- memory:', torch.cuda.memory_allocated(1)/(1024**3))

#--- memory: 0.0
#--- memory: 0.0


In [7]:
import importlib
importlib.reload(load_data)

<module 'utils.load_data' from '/home/u20/tnguyen9210/tnn1/LLMs/llm-reasoning-methods/utils/load_data.py'>

In [8]:
config = Config()

#  load data 
data_by_levels = load_data_prm800k(data_dir)
# print(data_by_levels)

# print(config.dataset_name)
orig_dataset = load_dataset(config.dataset_name, split='test', cache_dir=data_dir)
print(orig_dataset)
# stop
# for data in dataset:
#     pprint.pprint(data)
#     stop
# ds_completions = load_completions(completions_dir)

# load random_seeds     
# random_seeds = np.loadtxt("random_seeds.txt").astype("int64")
# random_seeds = [int(seed) for seed in random_seeds]

1: 43
2: 90
3: 105
4: 128
5: 134
Dataset({
    features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id'],
    num_rows: 500
})


In [9]:
stop

NameError: name 'stop' is not defined

In [11]:
# general params
config = Config()
config.agg_strategy = 'last'
config.n = 16
config.beam_width = 4
config.lookahead = 0
config.num_iterations = 40
config.sort_completed = False
config.seed = 0
config.version = "v21"

# diverse_select params
config.lam = 10
config.normalize_embeds = True

config.num_proc = 12 

dataset_id = "tnguyen9210/LLM-Reasoning-Math-500"
level = 4
num_questions = len(data_by_levels[level])
# num_questions = 2
num_trials = 5
print(f"num_questions = {num_questions}")

# get batch of questions
batch_of_questions = [data_by_levels[level][q_idx]['problem'] for q_idx in range(num_questions)]
orig_dataset_by_level = orig_dataset.filter(lambda example: example['level'] == int(level))
orig_dataset_by_level = orig_dataset_by_level.select(range(num_questions))
print(orig_dataset_by_level)
# for data in orig_dataset_by_level:
#     print(data)

# load completions
# config_name = f"sd--n-{config.n}--bw-{config.beam_width}--depth-{config.num_iterations}--lam-{config.lam}--{config.normalize_embeds}--seed-{config.seed}--level-{level}--{config.version}"
config_name = "sd--n-16--bw-2--depth-2--lam-10--True--seed-0--level-4--v21"
config_name = "sd--n-8--bw-2--d-2--lam-10--True--level-4--v21"
config_name = "sd--n-8--bw-2--d-2--lam-10--True--level-4--v21"
config_name = "sd--n-8--bw-2--d-10--lam-10--True--level-4--v11"

completions_dir = f"results/generate_{config_name}.jsonl"
scores_dir = f"results/scores_{config_name}.jsonl"
start_idx = 0

# compute results
start_time = time.time()    
with open(completions_dir, 'r', encoding = 'utf-8') as fin:
    trial_idx = 0
    for line in fin:
        
        if trial_idx >= num_trials:
            break

        if trial_idx < start_idx:
            trial_idx += 1
            continue
            
        trial_data = json.loads(line)
        # print(len(trial_data["completions"][0]))
        # stop
        completions = [trial_data["completions"][q_idx] for q_idx in range(num_questions)]
        
        print(f"{len(completions)}")
        print(f"len = {len(completions[0])}")
        # print(batch_of_questions)
        batch_scores = []
        for q_idx in range(len(batch_of_questions)):
            print(f"q_idx: {q_idx} - {len(completions[q_idx])}")
            # print([batch_of_questions[q_idx]])
            # print([completions[q_idx]])
            scores = prm.score([batch_of_questions[q_idx]], [completions[q_idx]])
            batch_scores += scores
            
        # agg_scores = [
        #     [aggregate_scores(s, config.agg_strategy) for s in score] for score in batch_scores
        # ]
        print(len(batch_scores))
        print(len(batch_scores[0]))
        
        _orig_dataset_by_level = orig_dataset_by_level.add_column("completions", completions)
        _orig_dataset_by_level = _orig_dataset_by_level.add_column("scores", batch_scores)
        print(_orig_dataset_by_level)

        # for data in orig_dataset_by_level:
        #     print(data.keys())
            # print(data["completions"])
                
        _orig_dataset_by_level = score(_orig_dataset_by_level, config)
        # for data in orig_dataset_by_level:
        #     print(data)
        #     stop

        # _orig_dataset_by_level.push_to_hub(dataset_id, config_name=f"{config_name}--trial-{trial_idx}", split='test')
        _orig_dataset_by_level.to_json(f"results/{config_name}--nolimit--trial-{trial_idx}.jsonl")
        
        # compute the time
        if trial_idx % 1 == 0:
            total_time = time.time() - start_time
            time_per_trial = total_time/(trial_idx+1)
            time_per_question = time_per_trial/num_questions
            print(f"trial {trial_idx}")
            print(f"it takes {time_per_question:0.4f}s per question")
            print(f"it takes {time_per_trial:0.4f}s per trial")

        trial_idx += 1
        

total_time = time.time() - start_time
print(f"it takes {total_time:0.4f}s in total")

num_questions = 128
Dataset({
    features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id'],
    num_rows: 128
})
128
len = 16
q_idx: 0 - 16
q_idx: 1 - 9
q_idx: 2 - 21
q_idx: 3 - 14
q_idx: 4 - 14
q_idx: 5 - 11
q_idx: 6 - 13
q_idx: 7 - 14
q_idx: 8 - 27
q_idx: 9 - 18
q_idx: 10 - 17
q_idx: 11 - 12
q_idx: 12 - 12
q_idx: 13 - 14
q_idx: 14 - 15
q_idx: 15 - 18
q_idx: 16 - 10
q_idx: 17 - 14
q_idx: 18 - 16
q_idx: 19 - 14
q_idx: 20 - 12
q_idx: 21 - 24
q_idx: 22 - 8
q_idx: 23 - 11
q_idx: 24 - 13
q_idx: 25 - 16
q_idx: 26 - 19
q_idx: 27 - 14
q_idx: 28 - 10
q_idx: 29 - 11
q_idx: 30 - 14
q_idx: 31 - 14
q_idx: 32 - 27
q_idx: 33 - 18
q_idx: 34 - 13
q_idx: 35 - 22
q_idx: 36 - 20
q_idx: 37 - 20
q_idx: 38 - 12
q_idx: 39 - 12
q_idx: 40 - 11
q_idx: 41 - 17
q_idx: 42 - 16
q_idx: 43 - 18
q_idx: 44 - 10
q_idx: 45 - 8
q_idx: 46 - 9
q_idx: 47 - 12
q_idx: 48 - 10
q_idx: 49 - 23
q_idx: 50 - 22
q_idx: 51 - 24
q_idx: 52 - 20
q_idx: 53 - 14
q_idx: 54 - 14
q_idx: 55 - 12
q_idx: 56 - 16
q_idx: 57 - 1

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

[4, 8, 16]


Computing majority & weighted predictions:   0%|          | 0/3 [00:00<?, ?it/s]

Subsample 4 (num_proc=12):   0%|          | 0/128 [00:00<?, ? examples/s]

...

Compute naive pred 16 (num_proc=12):   0%|          | 0/128 [00:00<?, ? examples/s]

Computing majority & weighted predictions: 100%|██████████| 3/3 [00:11<00:00,  3.94s/it]


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

trial 0
it takes 4.4147s per question
it takes 565.0823s per trial
128
len = 12
q_idx: 0 - 12
q_idx: 1 - 25
q_idx: 2 - 28
q_idx: 3 - 24
q_idx: 4 - 14
q_idx: 5 - 15
q_idx: 6 - 17
q_idx: 7 - 14
q_idx: 8 - 31
q_idx: 9 - 16
q_idx: 10 - 25
q_idx: 11 - 10
q_idx: 12 - 17
q_idx: 13 - 16
q_idx: 14 - 22
q_idx: 15 - 14
q_idx: 16 - 10
q_idx: 17 - 14
q_idx: 18 - 22
q_idx: 19 - 18
q_idx: 20 - 14
q_idx: 21 - 12
q_idx: 22 - 10
q_idx: 23 - 9
q_idx: 24 - 14
q_idx: 25 - 8
q_idx: 26 - 17
q_idx: 27 - 16
q_idx: 28 - 13
q_idx: 29 - 15
q_idx: 30 - 14
q_idx: 31 - 18
q_idx: 32 - 10
q_idx: 33 - 17
q_idx: 34 - 10
q_idx: 35 - 22
q_idx: 36 - 17
q_idx: 37 - 18
q_idx: 38 - 13
q_idx: 39 - 20
q_idx: 40 - 9
q_idx: 41 - 20
q_idx: 42 - 15
q_idx: 43 - 12
q_idx: 44 - 18
q_idx: 45 - 19
q_idx: 46 - 8
q_idx: 47 - 14
q_idx: 48 - 10
q_idx: 49 - 27
q_idx: 50 - 13
q_idx: 51 - 19
q_idx: 52 - 17
q_idx: 53 - 12
q_idx: 54 - 13
q_idx: 55 - 14
q_idx: 56 - 14
q_idx: 57 - 11
q_idx: 58 - 22
q_idx: 59 - 14
q_idx: 60 - 14
q_idx: 61 - 12
q_id

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.81 GiB. GPU 3 has a total capacity of 31.73 GiB of which 8.41 GiB is free. Including non-PyTorch memory, this process has 23.32 GiB memory in use. Of the allocated memory 15.28 GiB is allocated by PyTorch, and 7.67 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
dataset_id = "tnguyen9210/LLM-Reasoning-Math-500"
orig_dataset_by_level.push_to_hub(dataset_id, config_name=config_name, split='test')
orig_dataset_by_level.to_json(f"results/{config_name}.jsonl")